In [1]:
#generate the digit 5 from the MNIST dataset

from numpy import mean
from numpy import ones
from numpy import zeros
from numpy import expand_dims 
from numpy.random import randn
from numpy.random import randint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal

In [2]:
import matplotlib.pyplot as plt
from keras import backend
from keras.optimizers import RMSprop
from keras.datasets.mnist import load_data
from keras.constraints import Constraint


In [3]:
class ClipConstraint(Constraint):
  def __init__(self, clip_val):
    self.clip_val = clip_val

  def __call__(self, weights):
    return backend.clip(weights, -self.clip_val, self.clip_val)

  def get_config(self):
    return {'clip_value' : self.clip_val} 

In [4]:
def wasserstein_loss(y_true, y_pred):
  return backend.mean(y_true * y_pred)

In [5]:
def wasserstein_loss(y_true, y_pred):
  return backend.mean(y_true * y_pred)


#critic model
def define_critic(in_shape = (28, 28, 1)):
  w_init = RandomNormal(stddev = 0.02)
  clip_val = ClipConstraint(0.01) #avoids explosion of grads
  model = Sequential()
  model.add(Conv2D(64, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer=w_init,
                   kernel_constraint = clip_val, input_shape = in_shape))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))


  model.add(Conv2D(64, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer=w_init,
                   kernel_constraint = clip_val))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))  

  model.add(Flatten())
  model.add(Dense(1))  #can also use the linear actn for score, but same op
  optim = RMSprop(lr = 0.00005)
  model.compile(loss = wasserstein_loss, optimizer = optim)
  return model

In [6]:
def define_generator(latent_dims):
  w_init = RandomNormal(stddev=0.02)
  model = Sequential()
  n_nodes = 128 * 7 * 7
  model.add(Dense(n_nodes, kernel_initializer= w_init, input_dim = latent_dims))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Reshape((7, 7, 128)))

  model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer= w_init))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer= w_init))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))

  model.add(Conv2D(1, (7, 7), activation = 'tanh', padding = 'same', kernel_initializer= w_init))
  return model  

In [7]:
def wasserstein_loss(y_true, y_pred):
  return backend.mean(y_true * y_pred)
  
def define_WGAN(generator, critic):
  critic.trainable = False
  model = Sequential()
  model.add(generator)
  model.add(critic)
  optim = RMSprop(lr = 0.00005)
  model.compile(loss = wasserstein_loss, optimizer = optim)
  return model

In [8]:
def load_real_samples():
  (X_train, y_train), (X_test, y_test) = load_data()
  selected_idx = y_train == 5
  X = X_train[selected_idx]
  X = expand_dims(X, axis = -1)
  X = X.astype('float32')
  X = (X - 127.5)/127.5
  return X

In [9]:
def generate_real_samples(data, n_samples):
  idx = randint(0, data.shape[0], n_samples)
  X = data[idx]
  y = -ones((n_samples, 1))
  return X, y

In [10]:
def latent_points(latent_dims, n_samples):
  x_in = randn(latent_dims * n_samples)
  x_in = x_in.reshape(n_samples, latent_dims)
  return x_in

In [11]:
def generate_fake_samples(generator, latent_dims, n_samples):
  x_in = latent_points(latent_dims, n_samples)
  X = generator.predict(x_in)
  y = ones((n_samples, 1))
  return X, y

In [12]:
def summarize_performance(step, g_model, latent_dims, n_samples=100):
	X, _ = generate_fake_samples(g_model, latent_dims, n_samples)
	X = (X + 1) / 2.0
	for i in range(100):
		plt.subplot(10, 10, i+1)
		plt.axis('off')
		plt.imshow(X[i, :, :, 0], cmap='gray_r')
  
	filename1 = 'generated_plot_%04d.png' % (step+1)
	plt.savefig(filename1)
	plt.close()
 
	filename2 = 'model_%04d.h5' % (step+1)
	g_model.save(filename2)
	print('---> Saved: %s and %s' % (filename1, filename2))

In [13]:
def plot_history(c1, c2, g):
  plt.plot(c1, label = 'critic_real')
  plt.plot(c2, label = 'critic_fake')
  plt.plot(g, label = 'generated')
  plt.legend()
  plt.savefig('line_loss_plot.png')
  plt.close()

In [16]:
def train(generator, critic, wgan, data, latent_dims, n_epochs = 1000, n_batch = 64, n_critic = 5):
  batches_per_epoch = int(data.shape[0] / n_batch)
  n_steps = batches_per_epoch * n_epochs
  half_batch = int(n_batch / 2)
  c1, c2, g = list(), list(), list()
  for i in range(n_steps):
    c1_temp, c2_temp = list(), list()
    for j in range(n_critic):
      X_real, y_real = generate_real_samples(data, half_batch)
      c1_loss = critic.train_on_batch(X_real, y_real)
      c1_temp.append(c1_loss)
      X_fake, y_fake = generate_fake_samples(generator, latent_dims, half_batch)
      c2_loss = critic.train_on_batch(X_fake, y_fake)
      c2_temp.append(c2_loss)  
    c1.append(mean(c1_temp))
    c2.append(mean(c2_temp))
    X_g = latent_points(latent_dims, n_batch)
    y_g = -ones((n_batch, 1))  #real samples
    g_loss = wgan.train_on_batch(X_g, y_g)
    g.append(g_loss) 
    print('-->%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1[-1], c2[-1], g_loss))    #loss for that batch alone
    if (i+1) % batches_per_epoch == 0:
      summarize_performance(i, generator, latent_dims)

  plot_history(c1, c2, g)    

In [ ]:
latent_dims = 50
critic = define_critic()
generator = define_generator(latent_dims)
wgan = define_WGAN(generator, critic)
data = load_real_samples()
print(data.shape)
train(generator, critic, wgan, data, latent_dims)

Streaming output truncated to the last 5000 lines.
-->20176, c1=-201170.397, c2=-187043.413 g=-17330.391
-->20177, c1=-195696.528, c2=-184608.231 g=-36851.961
-->20178, c1=-198619.734, c2=-183758.944 g=-20106.910
-->20179, c1=-199090.122, c2=-183680.347 g=2574.353
-->20180, c1=-192919.472, c2=-181397.959 g=-32330.471
-->20181, c1=-197578.612, c2=-185651.041 g=-20777.965
-->20182, c1=-188574.300, c2=-179511.584 g=-58775.055
-->20183, c1=-200300.300, c2=-186147.728 g=-44779.695
-->20184, c1=-199289.122, c2=-185388.244 g=11072.441
-->20185, c1=-193721.872, c2=-182754.397 g=-8642.211
-->20186, c1=-201268.916, c2=-185392.478 g=-21751.447
-->20187, c1=-192094.525, c2=-181372.244 g=-50605.371
-->20188, c1=-198693.362, c2=-184776.953 g=-50519.547
-->20189, c1=-199428.847, c2=-184960.422 g=-20158.535
-->20190, c1=-193991.188, c2=-183839.947 g=-57140.711
-->20191, c1=-195898.528, c2=-185358.038 g=-44507.492
-->20192, c1=-201508.366, c2=-187679.788 g=-43509.195
-->20193, c1=-198645.769, c2=-18564